In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df=pd.read_csv('dataset/cleaned_data.csv')

In [3]:
df.head()

,date,state,area,number_of_monitoring_stations,prominent_pollutants,aqi_value,air_quality_status,unit
0,30-04-2025,Maharashtra,Amravati,2,PM10,78.0,Satisfactory,number_of_monitoring_stations in Absolute Numb...
1,30-04-2025,Bihar,Purnia,1,CO,56.0,Satisfactory,number_of_monitoring_stations in Absolute Numb...
2,30-04-2025,Madhya Pradesh,Katni,1,O3,98.0,Satisfactory,number_of_monitoring_stations in Absolute Numb...
3,30-04-2025,Chhattisgarh,Tumidih,1,PM10,103.0,Moderate,number_of_monitoring_stations in Absolute Numb...
4,30-04-2025,Assam,Byrnihat,1,PM2.5,61.0,Satisfactory,number_of_monitoring_stations in Absolute Numb...


In [4]:
df.shape

(235785, 8)

In [5]:
# AQI Irrevesible Degration
import pandas as pd
from scipy.stats import linregress

df['date'] = pd.to_datetime(df['date'])

# Define Tier 1 and Tier 2 cities
tier1 = ['Delhi', 'Mumbai', 'Bengaluru', 'Chennai', 'Hyderabad', 'Kolkata', 'Ahmedabad', 'Pune']
tier2 = ['Jaipur', 'Lucknow', 'Indore', 'Bhopal', 'Surat', 'Patna', 'Nagpur', 'Kanpur']
cities = tier1 + tier2

results = []

# Loop through cities and compute AQI trend (slope)
for city in cities:
    city_df = df[df['area'] == city].copy()
    city_df = city_df.resample('M', on='date').mean(numeric_only=True)

    if len(city_df) < 6 or city_df['aqi_value'].isnull().all():
        continue  # Skip if not enough data

    x = range(len(city_df))
    y = city_df['aqi_value'].values

    # Check if there's any NaN in y
    if pd.isna(y).any():
        continue

    slope, intercept, r_value, p_value, std_err = linregress(x, y)
    results.append((city, round(slope, 2), round(r_value**2, 2)))


C:\Users\aky\AppData\Local\Temp\ipykernel_4484\3970420984.py:5: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['date'] = pd.to_datetime(df['date'])


In [6]:
# Define slope threshold for irreversible degradation
threshold = 0.5  # you can adjust this (e.g., 0.5 or 1 AQI unit/month)

# Display results
print("City-wise AQI Slope Trend:")
for city, slope, r2 in results:
    print(f"{city}: Slope={slope}, R²={r2}")

# Extract priority cities
priority_cities = [city for city, slope, _ in results if slope > threshold]

print("\n🚨 Priority Cities (Irreversible AQI Degradation):")
print(priority_cities)

City-wise AQI Slope Trend:
Delhi: Slope=0.58, R²=0.01
Mumbai: Slope=-0.43, R²=0.01
Bengaluru: Slope=0.15, R²=0.01
Chennai: Slope=-0.01, R²=0.0
Hyderabad: Slope=0.04, R²=0.0
Kolkata: Slope=0.4, R²=0.01
Ahmedabad: Slope=0.35, R²=0.02
Jaipur: Slope=0.58, R²=0.02
Lucknow: Slope=0.26, R²=0.0
Indore: Slope=-0.7, R²=0.05
Bhopal: Slope=0.14, R²=0.0
Surat: Slope=-3.32, R²=0.26
Patna: Slope=0.2, R²=0.0
Nagpur: Slope=-0.0, R²=0.0
Kanpur: Slope=-0.21, R²=0.0

🚨 Priority Cities (Irreversible AQI Degradation):
['Delhi', 'Jaipur']


In [7]:
import pandas as pd
from scipy.stats import linregress
df['date'] = pd.to_datetime(df['date'])

# Define Tier 1 & Tier 2 cities
tier1 = ['Delhi', 'Mumbai', 'Bengaluru', 'Chennai', 'Hyderabad', 'Kolkata', 'Ahmedabad', 'Pune']
tier2 = ['Jaipur', 'Lucknow', 'Indore', 'Bhopal', 'Surat', 'Patna', 'Nagpur', 'Kanpur']
target_areas = tier1 + tier2

results = []

# Calculate slope and R² for each area
for area in target_areas:
    area_df = df[df['area'] == area].copy()
    area_df = area_df.resample('M', on='date').mean(numeric_only=True)

    if len(area_df) < 6 or area_df['aqi_value'].isnull().all():
        continue

    x = range(len(area_df))
    y = area_df['aqi_value'].values

    if pd.isna(y).any():
        continue

    slope, intercept, r_value, p_value, std_err = linregress(x, y)
    results.append({'area': area, 'Slope': round(slope, 2), 'R2': round(r_value**2, 2)})

# Create final DataFrame
trend_df = pd.DataFrame(results)

# Display result
print(trend_df)


         area  Slope    R2
0       Delhi   0.58  0.01
1      Mumbai  -0.43  0.01
2   Bengaluru   0.15  0.01
3     Chennai  -0.01  0.00
4   Hyderabad   0.04  0.00
5     Kolkata   0.40  0.01
6   Ahmedabad   0.35  0.02
7      Jaipur   0.58  0.02
8     Lucknow   0.26  0.00
9      Indore  -0.70  0.05
10     Bhopal   0.14  0.00
11      Surat  -3.32  0.26
12      Patna   0.20  0.00
13     Nagpur  -0.00  0.00
14     Kanpur  -0.21  0.00


In [8]:
trend_df.to_csv('dataset/tier1_2_cities.cvs',index=False)

In [4]:
idsp=pd.read_csv('dataset/idsp.csv',encoding='latin')

In [10]:
idsp.head()

,year,week,outbreak_starting_date,reporting_date,state,district,disease_illness_name,status,cases,deaths,unit,note
0,2025,16,15-04-2025,15-04-2025,Andhra Pradesh,Kakinada,Acute Diarrheal Disease,Reported,22,0,"cases in absolute number, deaths in absolute n...",NaN
1,2025,16,15-04-2025,17-04-2025,Assam,Biswanath,Chickenpox,Reported,1,1,"cases in absolute number, deaths in absolute n...",NaN
2,2025,16,19-04-2025,20-04-2025,Assam,Dhemaji,Food Poisoning,Reported,16,0,"cases in absolute number, deaths in absolute n...",NaN
3,2025,16,19-04-2025,19-04-2025,Bihar,Gopalganj,Fever with Rash,Reported,5,0,"cases in absolute number, deaths in absolute n...",NaN
4,2025,16,12-04-2025,15-04-2025,Bihar,Madhubani,Acute Diarrheal Disease,Reported,21,0,"cases in absolute number, deaths in absolute n...",NaN


In [11]:
idsp.shape

(6474, 12)

In [12]:
idsp['disease_illness_name'].value_counts()

disease_illness_name
Acute Diarrheal Disease           1967
Food Poisoning                     812
Dengue                             509
Chickenpox                         439
Hepatitis A                        385
                                  ... 
Leptospirosis and Scrub Typhus       1
Fever with Altered sensandium        1
Tetanus                              1
Fever of Unknown Cause               1
Malaria (Plasmodium Vivax)           1
Name: count, Length: 127, dtype: int64

In [16]:
# Pollution Emergencies with increase purifier searches
!pip install pytrends
from pytrends.request import TrendReq
pytrends = TrendReq()
pytrends.build_payload(kw_list=["air purifier"], geo='IN', timeframe='today 5-y')
trends_df = pytrends.interest_over_time()

In [17]:
trends_df=trends_df.reset_index()

In [18]:
trends_df

,date,air purifier,isPartial
0,2020-07-26,11,False
1,2020-08-02,11,False
2,2020-08-09,10,False
3,2020-08-16,9,False
4,2020-08-23,10,False
...,...,...,...
257,2025-06-29,8,False
258,2025-07-06,8,False
259,2025-07-13,10,False
260,2025-07-20,9,False


In [19]:
trends_df.to_csv('dataset/air_purifier_searches.csv',index=False)

In [6]:
import pandas as pd

# Load AQI and IDSP datasets
aqi_df = df.copy()
idsp_df = idsp.copy()

# Rename for consistency
aqi_df['date'] = pd.to_datetime(aqi_df['date'])
idsp_df['reporting_date'] = pd.to_datetime(idsp_df['reporting_date'])

# Add 'month' column to both for aggregation
aqi_df['month'] = aqi_df['date'].dt.to_period('M').astype(str)
idsp_df['month'] = idsp_df['reporting_date'].dt.to_period('M').astype(str)




C:\Users\aky\AppData\Local\Temp\ipykernel_4188\3959165481.py:8: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  aqi_df['date'] = pd.to_datetime(aqi_df['date'])
C:\Users\aky\AppData\Local\Temp\ipykernel_4188\3959165481.py:9: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  idsp_df['reporting_date'] = pd.to_datetime(idsp_df['reporting_date'])


In [7]:
# Filter for pediatric asthma illness (we assume it's named something like 'asthma')
asthma_df = idsp_df[idsp_df['disease_illness_name'].str.contains('asthma', case=False, na=False)]

# Group AQI by State and Month
monthly_aqi = aqi_df.groupby(['state', 'month'])['aqi_value'].mean().reset_index()



In [8]:
asthma_df

,year,week,outbreak_starting_date,reporting_date,state,district,disease_illness_name,status,cases,deaths,unit,note,month


In [ ]:
# Group Asthma cases by State and Month
asthma_cases = asthma_df.groupby(['state', 'month'])['cases'].sum().reset_index()

# Standardize column names for merging
asthma_cases.rename(columns={'state': 'State'}, inplace=True)

# Merge AQI and disease data
merged_df = pd.merge(monthly_aqi, asthma_cases, on=['State', 'month'])

# Drop missing values
merged_df.dropna(subset=['AQI', 'cases'], inplace=True)

# Convert cases to numeric if needed
merged_df['cases'] = pd.to_numeric(merged_df['cases'], errors='coerce')

# Calculate correlation
correlation = merged_df['AQI'].corr(merged_df['cases'])
print(f"\n📊 Correlation between AQI and pediatric asthma cases: {round(correlation, 3)}")

# Optional: Display merged data
print("\n🧾 Merged Dataset Sample:")
print(merged_df.head())

In [18]:
idsp['disease_illness_name'].values

array(['Acute Diarrheal Disease', 'Chickenpox', 'Food Poisoning', ...,
       'Chickenpox', 'Food Poisoning', 'Food Poisoning'], dtype=object)

In [4]:
df.head()

,date,state,area,number_of_monitoring_stations,prominent_pollutants,aqi_value,air_quality_status,unit
0,30-04-2025,Maharashtra,Amravati,2,PM10,78.0,Satisfactory,number_of_monitoring_stations in Absolute Numb...
1,30-04-2025,Bihar,Purnia,1,CO,56.0,Satisfactory,number_of_monitoring_stations in Absolute Numb...
2,30-04-2025,Madhya Pradesh,Katni,1,O3,98.0,Satisfactory,number_of_monitoring_stations in Absolute Numb...
3,30-04-2025,Chhattisgarh,Tumidih,1,PM10,103.0,Moderate,number_of_monitoring_stations in Absolute Numb...
4,30-04-2025,Assam,Byrnihat,1,PM2.5,61.0,Satisfactory,number_of_monitoring_stations in Absolute Numb...


In [9]:
df['state'].value_counts().index.to_list()

['Rajasthan',
 'Karnataka',
 'Maharashtra',
 'Haryana',
 'Uttar Pradesh',
 'Bihar',
 'Madhya Pradesh',
 'Tamil Nadu',
 'Odisha',
 'Punjab',
 'Andhra Pradesh',
 'West Bengal',
 'Gujarat',
 'Kerala',
 'Chhattisgarh',
 'Assam',
 'Uttarakhand',
 'Delhi',
 'Chandigarh',
 'Telangana',
 'Puducherry',
 'Himachal Pradesh',
 'Tripura',
 'Mizoram',
 'Nagaland',
 'Meghalaya',
 'Sikkim',
 'Manipur',
 'Jharkhand',
 'Arunachal Pradesh',
 'Jammu and Kashmir',
 'Andaman and Nicobar Islands']

In [10]:
cities=pd.read_csv('dataset/Cities_with_population_annul.csv')

In [11]:
cities.head()

,City,Population_2025,Per_Capita_Income_₹
0,Delhi,32900000,461910
1,Mumbai,21700000,277603
2,Bengaluru,14600000,332926
3,Kolkata,15600000,154119
4,Chennai,12200000,315220


In [14]:
cities_name=cities['City'].value_counts().index.to_list()

In [15]:
cities_name

['Delhi',
 'Mumbai',
 'Noida',
 'Chandigarh',
 'Jodhpur',
 'Amritsar',
 'Thane',
 'Nashik',
 'Meerut',
 'Rajkot',
 'Surat',
 'Visakhapatnam',
 'Varanasi',
 'Agra',
 'Ludhiana',
 'Vadodara',
 'Patna',
 'Bhopal',
 'Indore',
 'Nagpur',
 'Kanpur',
 'Lucknow',
 'Jaipur',
 'Ahmedabad',
 'Pune',
 'Hyderabad',
 'Chennai',
 'Kolkata',
 'Bengaluru',
 'Ghaziabad']

In [16]:
df.head()

,date,state,area,number_of_monitoring_stations,prominent_pollutants,aqi_value,air_quality_status,unit
0,30-04-2025,Maharashtra,Amravati,2,PM10,78.0,Satisfactory,number_of_monitoring_stations in Absolute Numb...
1,30-04-2025,Bihar,Purnia,1,CO,56.0,Satisfactory,number_of_monitoring_stations in Absolute Numb...
2,30-04-2025,Madhya Pradesh,Katni,1,O3,98.0,Satisfactory,number_of_monitoring_stations in Absolute Numb...
3,30-04-2025,Chhattisgarh,Tumidih,1,PM10,103.0,Moderate,number_of_monitoring_stations in Absolute Numb...
4,30-04-2025,Assam,Byrnihat,1,PM2.5,61.0,Satisfactory,number_of_monitoring_stations in Absolute Numb...


In [31]:
df_new=df.copy()

In [32]:
df_new=df_new[df_new['area'].isin(cities_name)]

In [33]:
df_new=df_new.groupby('area')['aqi_value'].mean().reset_index()

In [34]:
df_new.rename(columns={'area':'City'},inplace=True)

In [35]:
merged_df = pd.merge(df_new, cities, on="City", how="left")

In [36]:
merged_df.head()

,City,aqi_value,Population_2025,Per_Capita_Income_₹
0,Agra,84.222667,2500000,104126
1,Ahmedabad,113.584746,8800000,297722
2,Amritsar,121.214217,1600000,196505
3,Bengaluru,74.885333,14600000,332926
4,Bhopal,118.384201,2700000,142565


In [37]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City                 27 non-null     object 
 1   aqi_value            27 non-null     float64
 2   Population_2025      27 non-null     int64  
 3   Per_Capita_Income_₹  27 non-null     int64  
dtypes: float64(1), int64(2), object(1)
memory usage: 996.0+ bytes


In [38]:
merged_df['City_Risk_Scaled'] = (merged_df['aqi_value'] *
                          merged_df['Population_2025'] *
                          merged_df['Per_Capita_Income_₹']) / 1e9

In [39]:
merged_df.head()

,City,aqi_value,Population_2025,Per_Capita_Income_₹,City_Risk_Scaled
0,Agra,84.222667,2500000,104126,21924.423473
1,Ahmedabad,113.584746,8800000,297722,297586.763566
2,Amritsar,121.214217,1600000,196505,38110.719570
3,Bengaluru,74.885333,14600000,332926,363996.607486
4,Bhopal,118.384201,2700000,142565,45569.097792


In [40]:
merged_df.to_csv('Scaled_City_Risk.csv',index=False)

In [43]:
idsp.head()

,year,week,outbreak_starting_date,reporting_date,state,district,disease_illness_name,status,cases,deaths,unit,note
0,2025,16,15-04-2025,15-04-2025,Andhra Pradesh,Kakinada,Acute Diarrheal Disease,Reported,22,0,"cases in absolute number, deaths in absolute n...",NaN
1,2025,16,15-04-2025,17-04-2025,Assam,Biswanath,Chickenpox,Reported,1,1,"cases in absolute number, deaths in absolute n...",NaN
2,2025,16,19-04-2025,20-04-2025,Assam,Dhemaji,Food Poisoning,Reported,16,0,"cases in absolute number, deaths in absolute n...",NaN
3,2025,16,19-04-2025,19-04-2025,Bihar,Gopalganj,Fever with Rash,Reported,5,0,"cases in absolute number, deaths in absolute n...",NaN
4,2025,16,12-04-2025,15-04-2025,Bihar,Madhubani,Acute Diarrheal Disease,Reported,21,0,"cases in absolute number, deaths in absolute n...",NaN


In [45]:
idsp['disease_illness_name'].value_counts().index.to_list()

['Acute Diarrheal Disease',
 'Food Poisoning',
 'Dengue',
 'Chickenpox',
 'Hepatitis A',
 'Measles',
 'Mumps',
 'Cholera',
 'Fever with Rash',
 'Malaria',
 'Human Rabies',
 'Chikungunya',
 'Typhoid',
 'Scrub Typhus',
 'Japanese Encephalitis',
 'Leptospirosis',
 'Fever',
 'Acute Gastroenteritis',
 'Zika Virus',
 'Jaundice',
 'Diphtheria',
 'Leishmaniasis',
 'Mushroom Poisoning',
 'Animal bite - Dog Bite',
 'Food poisoning',
 'Acute Encephalitic Syndrome',
 'Shigellosis',
 'Kyasanur Fandest Disease',
 'West Nile Fever',
 'Mpox',
 'Dengue and Chikungunya',
 'Monkey Pox',
 'Crimean Congo Haemandrhagic Fever (CCHF)',
 'Pyrexia of Unknown andigin (PUO)',
 'Rubella',
 'Measles and Rubella',
 'Hepatitis A and E',
 'Fever of Unknown andigin (PUO)',
 'Dysentery',
 'Anthrax',
 'Paratyphoid',
 'Hepatitis',
 'Hand Foot and Mouth Disease',
 'Acute Hepatitis A',
 'Adenovirus',
 'Cutaneous Leishmaniasis',
 'Suspected Dengue Fever',
 'Acute Gastroenteritis (Norovirus)',
 'Gastroenteritis',
 'Nandovirus

In [54]:
asthma_related_diseases = [
    'Fever and Upper Respiratandy Tract Infection (URTI)',
    'Acute Respiratandy Illness',
    'Influenza A',
    'Seasonal Influenza',
    'ARI Influenza Like Illness(ILI)',
    'Influenza'
]

In [55]:
idsp.head()

,year,week,outbreak_starting_date,reporting_date,state,district,disease_illness_name,status,cases,deaths,unit,note
0,2025,16,15-04-2025,15-04-2025,Andhra Pradesh,Kakinada,Acute Diarrheal Disease,Reported,22,0,"cases in absolute number, deaths in absolute n...",NaN
1,2025,16,15-04-2025,17-04-2025,Assam,Biswanath,Chickenpox,Reported,1,1,"cases in absolute number, deaths in absolute n...",NaN
2,2025,16,19-04-2025,20-04-2025,Assam,Dhemaji,Food Poisoning,Reported,16,0,"cases in absolute number, deaths in absolute n...",NaN
3,2025,16,19-04-2025,19-04-2025,Bihar,Gopalganj,Fever with Rash,Reported,5,0,"cases in absolute number, deaths in absolute n...",NaN
4,2025,16,12-04-2025,15-04-2025,Bihar,Madhubani,Acute Diarrheal Disease,Reported,21,0,"cases in absolute number, deaths in absolute n...",NaN


In [58]:
idsp_new=idsp[idsp['disease_illness_name'].isin(asthma_related_diseases)]

In [59]:
idsp_new.head()

,year,week,outbreak_starting_date,reporting_date,state,district,disease_illness_name,status,cases,deaths,unit,note
2004,2024,30,23-07-2024,24-07-2024,Kerala,Ernakulam,ARI Influenza Like Illness(ILI),Reported in Same Week,19,0,"cases in absolute number, deaths in absolute n...",NaN
2643,2024,22,27-05-2024,28-05-2024,Karnataka,Hassan,Influenza,Reported in Same Week,1,1,"cases in absolute number, deaths in absolute n...",NaN
5553,2022,52,26-12-2022,26-12-2022,Karnataka,Tumakuru,Fever and Upper Respiratandy Tract Infection (...,Reported in Same Week,25,0,"cases in absolute number, deaths in absolute n...",NaN
5560,2022,52,10-12-2022,22-12-2022,Karnataka,Tumakuru,Fever and Upper Respiratandy Tract Infection (...,Reported in Same Week,16,0,"cases in absolute number, deaths in absolute n...",NaN
5577,2022,51,10-12-2022,22-12-2022,Karnataka,Tumakuru,Fever and Upper Respiratandy Tract Infection (...,Reported in Same Week,16,0,"cases in absolute number, deaths in absolute n...",NaN


In [61]:
aqi_df

,date,state,area,number_of_monitoring_stations,prominent_pollutants,aqi_value,air_quality_status,unit,month
0,2025-04-30,Maharashtra,Amravati,2,PM10,78.0,Satisfactory,number_of_monitoring_stations in Absolute Numb...,2025-04
1,2025-04-30,Bihar,Purnia,1,CO,56.0,Satisfactory,number_of_monitoring_stations in Absolute Numb...,2025-04
2,2025-04-30,Madhya Pradesh,Katni,1,O3,98.0,Satisfactory,number_of_monitoring_stations in Absolute Numb...,2025-04
3,2025-04-30,Chhattisgarh,Tumidih,1,PM10,103.0,Moderate,number_of_monitoring_stations in Absolute Numb...,2025-04
4,2025-04-30,Assam,Byrnihat,1,PM2.5,61.0,Satisfactory,number_of_monitoring_stations in Absolute Numb...,2025-04


In [5]:
import pandas as pd

# Load AQI and IDSP datasets
aqi_df = df.copy()  # Use full dataset
idsp_df = idsp.copy()

# Rename columns for consistency
aqi_df.rename(columns={'City': 'area', 'Date': 'date', 'State': 'state', 'AQI': 'aqi_value'}, inplace=True)
aqi_df['date'] = pd.to_datetime(aqi_df['date'], dayfirst=True)
idsp_df['reporting_date'] = pd.to_datetime(idsp_df['reporting_date'], dayfirst=True)

# Add 'month' column
aqi_df['month'] = aqi_df['date'].dt.to_period('M').astype(str)
idsp_df['month'] = idsp_df['reporting_date'].dt.to_period('M').astype(str)

# Define asthma-related diseases
asthma_related_diseases = [
    'Fever and Upper Respiratandy Tract Infection (URTI)',
    'Acute Respiratandy Illness',
    'Influenza A',
    'Seasonal Influenza',
    'ARI Influenza Like Illness(ILI)',
    'Influenza'
]

# Filter IDSP for asthma proxies (after month column is added)
asthma_df = idsp_df[idsp_df['disease_illness_name'].isin(asthma_related_diseases)]

# Group AQI and illness cases
monthly_aqi = aqi_df.groupby(['state', 'month'])['aqi_value'].mean().reset_index()
asthma_cases = asthma_df.groupby(['state', 'month'])['cases'].sum().reset_index()

# Merge
merged_df = pd.merge(monthly_aqi, asthma_cases, on=['state', 'month'])

# Clean and analyze
merged_df.dropna(subset=['aqi_value', 'cases'], inplace=True)
merged_df['cases'] = pd.to_numeric(merged_df['cases'], errors='coerce')

# Correlation
correlation = merged_df['aqi_value'].corr(merged_df['cases'])
print(f"\n📊 Correlation between AQI and asthma-proxy illness cases: {round(correlation, 3)}")

# Preview
print("\n🧾 Merged Dataset Sample:")
print(merged_df.head())



📊 Correlation between AQI and asthma-proxy illness cases: 0.692

🧾 Merged Dataset Sample:
               state    month  aqi_value  cases
0  Jammu and Kashmir  2022-06  75.321429     32
1  Jammu and Kashmir  2022-09  47.785714     18
2          Karnataka  2022-12  77.163043     57
3          Karnataka  2024-05  61.261119      1
4             Kerala  2024-07  54.771930     19


In [6]:
merged_df.head()

,state,month,aqi_value,cases
0,Jammu and Kashmir,2022-06,75.321429,32
1,Jammu and Kashmir,2022-09,47.785714,18
2,Karnataka,2022-12,77.163043,57
3,Karnataka,2024-05,61.261119,1
4,Kerala,2024-07,54.771930,19


In [7]:
merged_df.shape

(5, 4)

In [8]:
merged_df.to_csv('dataset/Aqi_Cases_correlation.csv',index=False)